# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
sys.path.append("../object_detection/")
from utils import label_map_util

from utils import visualization_utils as vis_util

../object_detection\utils\visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "c:\users\kofuji\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\kofuji\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\kofuji\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\kofuji\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\kofuji\appdata\local\p

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [12]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = os.path.join('bird_detection_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('kw_label_map.pbtxt')

NUM_CLASSES = 2

import os
cwd = os.getcwd()
print(cwd)

C:\Users\kofuji\Documents\Vienna\iot-ml\iotdeployTensorFlow


## Download Model

## Load a (frozen) Tensorflow model into memory.

In [13]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [14]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [15]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [16]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())
# saver = tf.train.import_meta_graph('./model.ckpt-3770.meta')
# saver.restore(sess,tf.train.latest_checkpoint('./'))
# graph = tf.get_default_graph()

# Detection

In [21]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'sampleimages'
# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'IMG_0010.JPG')]
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
print(TEST_IMAGE_PATHS)

['sampleimages\\IMG_0010.JPG']


In [33]:
import base64
import json
from PIL import Image
from io import BytesIO
# from skimage.color import rgb2gray
import numpy

# self.detection_graph = tf.Graph()
# with self.detection_graph.as_default():
#     od_graph_def = tf.GraphDef()
#     # Works up to here.
#     with tf.gfile.GFile(PATH_TO_MODEL, 'rb') as fid:
#         serialized_graph = fid.read()
#         od_graph_def.ParseFromString(serialized_graph)
#         tf.import_graph_def(od_graph_def, name='')
#     self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
#     self.d_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
#     self.d_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
#     self.d_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
#     self.num_d = self.detection_graph.get_tensor_by_name('num_detections:0')
# self.sess = tf.Session(graph=self.detection_graph)

sess = tf.Session()
# sess.run(tf.global_variables_initializer())
# # graph = tf.get_default_graph()
# detection_graph = tf.Graph()
# with detection_graph.as_default():
#     od_graph_def = tf.GraphDef()
#     with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
#         serialized_graph = fid.read()
#         od_graph_def.ParseFromString(serialized_graph)
#         tf.import_graph_def(od_graph_def, name='')

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
scores = detection_graph.get_tensor_by_name('detection_scores:0')
classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

image = Image.open(os.path.join(PATH_TO_TEST_IMAGES_DIR, 'IMG_0010.JPG'))
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)

output_dict = sess.run(
    [boxes, scores, classes, num_detections],
    feed_dict={image_tensor: image_np_expanded})

# all outputs are float32 numpy arrays, so convert types as appropriate
output_dict['num_detections'] = int(output_dict['num_detections'][0])
output_dict['detection_classes'] = output_dict[
    'detection_classes'][0].astype(np.uint8)
output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
output_dict['detection_scores'] = output_dict['detection_scores'][0]

# boxes, scores, classes, num_detections = map(
#     np.squeeze, [boxes, scores, classes, num_detections])

# json.dumps(output_dict)
print(output_dict)


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8) is not an element of this graph.

In [41]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      print(all_tensor_names)
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8).tolist()
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0].tolist()
      output_dict['detection_scores'] = output_dict['detection_scores'][0].tolist()
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [43]:
global sess

In [45]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

    # Get handles to input and output tensors
    ops = tf.get_default_graph().get_operations()
    all_tensor_names = {output.name for op in ops for output in op.outputs}
    tensor_dict = {}
    for key in [
        'num_detections', 'detection_boxes', 'detection_scores',
        'detection_classes', 'detection_masks'
    ]:
      tensor_name = key + ':0'
      if tensor_name in all_tensor_names:
        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
            tensor_name)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
    sess = tf.Session(graph=detection_graph)


In [47]:
image = Image.open(os.path.join(PATH_TO_TEST_IMAGES_DIR, 'IMG_0010.JPG'))
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)

# Run inference
output_dict = sess.run(tensor_dict,
                     feed_dict={image_tensor: image_np_expanded})

# all outputs are float32 numpy arrays, so convert types as appropriate
output_dict['num_detections'] = int(output_dict['num_detections'][0])
output_dict['detection_classes'] = output_dict[
  'detection_classes'][0].astype(np.uint8).tolist()
output_dict['detection_boxes'] = output_dict['detection_boxes'][0].tolist()
output_dict['detection_scores'] = output_dict['detection_scores'][0].tolist()
if 'detection_masks' in output_dict:
    output_dict['detection_masks'] = output_dict['detection_masks'][0]

print(json.dumps(output_dict))

{"num_detections": 173, "detection_boxes": [[0.02787705883383751, 0.5199805498123169, 0.08676732331514359, 0.5993831753730774], [0.3131874203681946, 0.3205956220626831, 0.40297380089759827, 0.3980810344219208], [0.07310472428798676, 0.14568670094013214, 0.18813197314739227, 0.2213737964630127], [0.24183334410190582, 0.23310595750808716, 0.32236817479133606, 0.3076658546924591], [0.48087620735168457, 0.30460846424102783, 0.5663688778877258, 0.3754548132419586], [0.2747368812561035, 0.3935248851776123, 0.3627084195613861, 0.46211108565330505], [0.46480652689933777, 0.47256824374198914, 0.5253232717514038, 0.536319375038147], [0.34587526321411133, 0.1055869460105896, 0.47226619720458984, 0.17017444968223572], [0.28794848918914795, 0.15942521393299103, 0.40056928992271423, 0.22536887228488922], [0.027194032445549965, 0.3940444886684418, 0.09812019765377045, 0.4482467472553253], [0.4637099802494049, 0.3851672410964966, 0.5506717562675476, 0.44126981496810913], [0.5007516145706177, 0.5701086

In [42]:
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.

  print(json.dumps(output_dict))
#   print(output_dict)
#   vis_util.visualize_boxes_and_labels_on_image_array(
#       image_np,
#       output_dict['detection_boxes'],
#       output_dict['detection_classes'],
#       output_dict['detection_scores'],
#       category_index,
#       instance_masks=output_dict.get('detection_masks'),
#       use_normalized_coordinates=True,
#       line_thickness=8)
#   plt.figure(figsize=IMAGE_SIZE)
#   plt.imshow(image_np)

{'SecondStagePostprocessor/BatchMultiClassNonMaxSuppression/map/while/Slice:0', 'BatchMultiClassNonMaxSuppression/map/while/PadOrClipBoxList/cond/cond/Merge:0', 'FirstStageFeatureExtractor/resnet_v1_101/block3/unit_21/bottleneck_v1/conv1/BatchNorm/gamma/read:0', 'FirstStageFeatureExtractor/resnet_v1_101/resnet_v1_101/block3/unit_18/bottleneck_v1/add:0', 'GridAnchorGenerator/Meshgrid_2/ExpandedShape_1/Slice_1:0', 'FirstStageFeatureExtractor/resnet_v1_101/block3/unit_5/bottleneck_v1/conv1/weights:0', 'GridAnchorGenerator/Meshgrid/ExpandedShape/ExpandDims/dim:0', 'GridAnchorGenerator/Meshgrid_3/Shape:0', 'Decode/get_center_coordinates_and_sizes/sub:0', 'FirstStageFeatureExtractor/resnet_v1_101/resnet_v1_101/block2/unit_4/bottleneck_v1/Pad/paddings:0', 'map/while/TensorArrayReadV3/Enter_1:0', 'FirstStageFeatureExtractor/resnet_v1_101/resnet_v1_101/block1/unit_2/bottleneck_v1/conv1/Relu:0', 'FirstStageFeatureExtractor/resnet_v1_101/block3/unit_8/bottleneck_v1/conv3/weights/read:0', 'FirstSt

{"num_detections": 173, "detection_boxes": [[0.02787705883383751, 0.5199805498123169, 0.08676732331514359, 0.5993831753730774], [0.3131874203681946, 0.3205956220626831, 0.40297380089759827, 0.3980810344219208], [0.07310472428798676, 0.14568670094013214, 0.18813197314739227, 0.2213737964630127], [0.24183334410190582, 0.23310595750808716, 0.32236817479133606, 0.3076658546924591], [0.48087620735168457, 0.30460846424102783, 0.5663688778877258, 0.3754548132419586], [0.2747368812561035, 0.3935248851776123, 0.3627084195613861, 0.46211108565330505], [0.46480652689933777, 0.47256824374198914, 0.5253232717514038, 0.536319375038147], [0.34587526321411133, 0.1055869460105896, 0.47226619720458984, 0.17017444968223572], [0.28794848918914795, 0.15942521393299103, 0.40056928992271423, 0.22536887228488922], [0.027194032445549965, 0.3940444886684418, 0.09812019765377045, 0.4482467472553253], [0.4637099802494049, 0.3851672410964966, 0.5506717562675476, 0.44126981496810913], [0.5007516145706177, 0.5701086